**Install Necessary pacakages **

In [0]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

c:\users\y437l\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\y437l\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
c:\users\y437l\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [0]:
#!rm train.csv
#!rm test.csv

upload your data train.csv and test.csv

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%ls /gdrive

## To delete. Reason --> done in above cell with URL
train_data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/tweet-sentiment-extraction/train.csv')
test_data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/tweet-sentiment-extraction/test.csv')
sub_df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/tweet-sentiment-extraction/sample_submission.csv')

In [0]:
pip install nltk

In [0]:
pip install simpletransformers

In [0]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [0]:
!sh setup.sh

Process doucments and transfer data to json 


### **Helper functions**

In [0]:
####find the the selected_text's start index in the orignial context (not being used)
####example:
#### a = 'Sooo SAD I will miss you here in San Diego!!!'，b = 'Sooo SAD'
#### find(a,b) = 0

def find_start(context_string1,selected_string2):
  try:
        list_of_text_chars = context_string1.split()
        list_of_selected_chars = selected_string2.split()
        #staring character:
        s_chars = list_of_selected_chars[0]
        #length of selected_chars:
        s_len = len(list_of_selected_chars)
        #find the start index:
        #potential_start = list_of_text_chars.index(s_chars)
        #start_index = find_start_index(list_of_text_chars,list_of_selected_chars,potential_start,s_chars,s_len)
        start_index = list_of_text_chars.index(s_chars,0)
        ###count number of characters before the index
        if start_index == 0:
          start = start_index
        else:
          sub_list = list_of_text_chars[:start_index]
          sum_chars = len(sub_list)
          for i in sub_list:
            sum_chars += len(i)
          start = sum_chars
        return start
  except:
        pass


In [0]:
######helper function to help find_start to scan through the orignial index and find the true start(not being used any longer )
def find_start_index(list_of_text_chars,list_of_selected_chars,potential_start,s_chars,s_len):
    if list_of_text_chars[potential_start:potential_start + s_len] == list_of_selected_chars:
        start_index = potential_start
        return start_index
    else:
        new_potential_star = list_of_text_chars[potential_start:].index(s_chars)
        find_start_index(list_of_text_chars,list_of_selected_chars,new_potential_star,s_chars,s_len)

In [0]:
###a function transfer current pd dataframe to dictionary format (not being used any longer)
def transfer(data):
  temp_list = []
  ID = 0
  for i in range(0,len(data)):
    temp_dict = {}
    qas_dict2 = {}
    qas_list = []
    answer_list = []
    answer_dict3 = {}
    temp_dict['context'] = data.iloc[i].text
    qas_dict2['question'] = data.iloc[i].question
    qas_dict2['id'] = str(ID)
    qas_dict2['is_impossible'] = False
    answer_dict3['text'] = data.iloc[i].answer
    #print(data.iloc[i].answer_start)
    answer_dict3['answer_start'] = int(data.iloc[i].answer_start)
    answer_list.append(answer_dict3)
    qas_dict2['answers'] = answer_list
    qas_list.append(qas_dict2)
    temp_dict['qas'] = qas_list
    temp_list.append(temp_dict)
    ID += 1
  return temp_list

In [0]:
#drop the blank for string start with space
#example: drop_blank(" SCREAAAAAAAAAAAAAAAAAAAAAAAAAAAAAM - never f") ->'SCREAAAAAAAAAAAAAAAAAAAAAAAAAAAAAM - never f'
def drop_blank(d):
  if str(d)[0] == ' ':
    return d[1:]
  else:
    return d
      

In [0]:
####transfer the evaluation dict to the final result in pandas dataframe
def trans_dict_to_pd(text2,eval_list):
  temp = pd.DataFrame(columns=['ID', 'Perdicted', 'truth'])
  for part in text2:
    if part == "correct_text":
      for i in text2[part].keys():
        #print(i)
        ID_value =i
        Perdict_value = text2[part][i]
        for x in eval_list:
            #print(x)
            if x['qas'][0]['id'] == i:
                Truth_value = x['context']
        temp_dict= {'ID':ID_value, 'Perdicted':Perdict_value, 'truth':Truth_value }
        temp_df = pd.DataFrame([temp_dict])
        temp=temp.append(temp_df)
    else:
      for i in text2[part].keys():
        ID_value = text2[part][i]['ID']
        Perdict_value = text2[part][i]['predicted']
        Truth_value = text2[part][i]['truth']
        temp_dict= {'ID':ID_value, 'Perdicted':Perdict_value, 'truth':Truth_value }
        temp_df = pd.DataFrame([temp_dict])
        temp=temp.append(temp_df)
    return temp

In [0]:
###define the jaccard measure function
def jaccard(str1, str2): 
    #print(str2)
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [0]:
###Transfer the perdiction result into pandas dataframe by select the perdiction with highest probability
def trans_dict_to_pd_test(result):
  temp = pd.DataFrame(columns=['ID', 'selected_text'])
  for i in result[0]:
    ID_value = int(i['id'])
    Perdict_value = i['answer'][0]
    temp_dict= {'ID':ID_value, 'selected_text':Perdict_value}
    temp_df = pd.DataFrame([temp_dict])
    temp=temp.append(temp_df)
  return temp

In [0]:
###Transfer the test data into list of dicts
def transfer_text(data):
  temp_list = []
  ID = 0
  for i in range(0,len(data)):
    temp_dict = {}
    qas_dict2 = {}
    qas_list = []
    temp_dict['context'] = data.iloc[i].text
    qas_dict2['question'] = data.iloc[i].sentiment
    qas_dict2['id'] = str(ID)
    qas_list.append(qas_dict2)
    temp_dict['qas'] = qas_list
    temp_list.append(temp_dict)
    ID += 1
  return temp_list

In [0]:
def replace_the_wrong_pontuation(a,target,subsitue):
    if target in a:
      a = a.replace(target, subsitue)
      return a
    else:
      return a

In [0]:
## Adpated from https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing
def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

def do_qa_train(train):

    output = []
    for line in train:
        context = line[1]

        qas = []
        question = line[-1]
        qid = line[0]
        answers = []
        answer = line[2]
        if type(answer) != str or type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answer_starts = find_all(context, answer)
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer.lower()})
            break
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        output.append({'context': context.lower(), 'qas': qas})
        
    return output


In [0]:
def do_qa_test(test):
    output = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output


##Main Flow

In [0]:
import pandas as pd
import numpy as np

In [0]:
train_data = pd.read_csv('train.csv')
train_data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [0]:
####spilt data the train_df and eval_df
from sklearn.model_selection import train_test_split
train_df,eval_df = train_test_split(train_data,test_size=0.2, random_state=42)
train = np.array(train_df)
evalu = np.array(eval_df)

In [0]:
training_list = do_qa_train(train)

nan <class 'float'>
nan <class 'float'>
neutral <class 'str'>


In [0]:
eval_list = do_qa_train(evalu)

In [0]:
eval_list[0]

{'context': ' woooooooooo   are you coming to nottingham at any point?  lovelovelove<3',
 'qas': [{'question': 'positive',
   'id': 'a7f72a928a',
   'is_impossible': False,
   'answers': [{'answer_start': 55, 'text': 't?  lovelovelove'}]}]}

In [0]:
from simpletransformers.question_answering import QuestionAnsweringModel
import json
import os
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [0]:
####define training arguments
train_args = {
    'learning_rate': 3e-5,
    'num_train_epochs': 2,
    'max_seq_length': 384,
    'doc_stride': 128,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'train_batch_size': 2,
    'gradient_accumulation_steps': 8,
}

# **roberta**

In [0]:
###call the roberta model
model = QuestionAnsweringModel('roberta', 'roberta-base', args=train_args)

In [0]:
model.train_model(qa_train)


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|███████████████████████████████████████████████████████████████████████████| 21983/21983 [00:22<00:00, 989.00it/s]


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 5.840650

c:\users\y437l\appdata\local\programs\python\python36\lib\site-packages\torch\optim\lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 5.599775Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 3.694575Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 2.508857Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 2.181324Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 1.432607Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Running loss: 2.132376Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Running loss: 2.698130Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Running loss: 2.429947Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 256.0
Running loss: 1.109427Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Running loss: 1.393421Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale t

INFO:simpletransformers.question_answering.question_answering_model: Training of roberta model complete. Saved to outputs/.


In [0]:
##get evaulate result
result, text = model.eval_model(eval_list)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|████████████████████████████████████████████████████████████████████████████| 5497/5497 [00:04<00:00, 1102.00it/s]


In [0]:
###evaulate list example
text['correct_text']['ef42dee96c']

'resting had a whole day of walking'

# **Albert**

In [0]:
model2 = QuestionAnsweringModel('albert', 'albert-base-v2', args=train_args)

In [0]:
model2.train_model(training_list)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|███████████████████████████████████████████████████████████████████████████| 21983/21983 [00:23<00:00, 930.54it/s]


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 5.908095

c:\users\y437l\appdata\local\programs\python\python36\lib\site-packages\torch\optim\lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 6.019498Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 5.910810Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 5.852949Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 5.822381Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 5.797133Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Running loss: 0.613217


Running loss: 1.269851Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Running loss: 0.522537Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.685771Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.684634Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.594295Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.820680

INFO:simpletransformers.question_answering.question_answering_model: Training of albert model complete. Saved to outputs/.


In [0]:
result2, text2 = model2.eval_model(eval_list)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|████████████████████████████████████████████████████████████████████████████| 5497/5497 [00:05<00:00, 1027.35it/s]


# **distilbert**

In [0]:
model3 = QuestionAnsweringModel('distilbert', 'distilbert-base-cased', args=train_args)

In [0]:
model3.train_model(training_list)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|███████████████████████████████████████████████████████████████████████████| 21983/21983 [00:28<00:00, 784.30it/s]


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 5.325376Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 5.264580Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 4.718556Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.895140Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Running loss: 2.385106Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.537169


Running loss: 0.584578Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.061847Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.308324Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.671552


INFO:simpletransformers.question_answering.question_answering_model: Training of distilbert model complete. Saved to outputs/.


In [0]:
###evaluation
result3, text3 = model3.eval_model(eval_list)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|█████████████████████████████████████████████████████████████████████████████| 5497/5497 [00:05<00:00, 918.77it/s]


# Testing

In [0]:
test_data = pd.read_csv('test.csv')
test_data.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [0]:
#transfer test data to test list
test_data1 = np.array(test_data)
test_list = do_qa_test(test_data1)

In [0]:
predictions = model.predict(test_list)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|████████████████████████████████████████████████████████████████████████████| 3534/3534 [00:03<00:00, 1015.32it/s]


In [0]:
predictions2 = model2.predict(test_list)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|████████████████████████████████████████████████████████████████████████████| 3534/3534 [00:03<00:00, 1061.23it/s]


In [0]:
predictions3 = model3.predict(test_list)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
100%|█████████████████████████████████████████████████████████████████████████████| 3534/3534 [00:03<00:00, 896.85it/s]


In [0]:
##perdiction result example
predictions[0]

{'id': 'f87dea47db',
 'answer': 'last session of the day http://twitpic.com/67ezh'}

In [0]:
for i in text2:
  print(i)

In [0]:
len(eval_list)

5497

# Evaluation

In [0]:
##The transfer albert's evaluation result into pandas datframe
albert_df = trans_dict_to_pd(text2,eval_list)

In [0]:
albert_df['j_score'] = albert_df.apply(lambda x: jaccard(x.truth, x.Perdicted), axis=1)

In [0]:
##The result of albert method

print(len(albert_df),',',albert_df['j_score'].mean())

2494 , 0.6988551523466628


In [0]:
##The transfer roberta's evaluation result into pandas datframe
roberta_df = trans_dict_to_pd(text,eval_list)

In [0]:
roberta_df['j_score'] = roberta_df.apply(lambda x: jaccard(x.truth, x.Perdicted), axis=1)

In [0]:
roberta_df

,ID,Perdicted,truth,j_score
0,ef42dee96c,resting had a whole day of walking,resting had a whole day of walking,1.000000
0,07d17131b1,"was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ...",1.000000
0,2820205db5,horrible.,i know! i`m so slow its horrible. don`t tell ...,0.090909
0,7d3ce4363c,glad,"glad i went out, glad i didn`t leave early, an...",0.058824
0,8f8a38f144,are the drugs working?,_lia are the drugs working?,0.800000
...,...,...,...,...
0,a4bb6136b2,sorry,sorry but there is no parking space. and i ju...,0.050000
0,8f4e7ea1da,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...,1.000000
0,b8ea6c30db,excited,bit excited are u bradie lol,0.166667
0,f7cbfff26e,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...,1.000000


In [0]:
print(len(roberta_df),roberta_df['j_score'].mean())

2477 0.695649215770056


In [0]:
##The transfer distilbert_df's evaluation result into pandas datframe
distilbert_df = trans_dict_to_pd(text3,eval_list)

In [0]:
distilbert_df['j_score'] = distilbert_df.apply(lambda x: jaccard(x.truth, x.Perdicted), axis=1)

In [0]:
print(len(distilbert_df),distilbert_df['j_score'].mean())

2472 0.7089540657786745


In [0]:
albert_df.to_csv('albert.csv')
roberta_df.to_csv('roberta.csv')
distilbert_df.to_csv('distilbert_.csv')

In [0]:
#perdiction example
predictions[0]

{'id': 'f87dea47db',
 'answer': 'last session of the day http://twitpic.com/67ezh'}

In [0]:
def trans_dict_to_pd_test(result):
  temp = pd.DataFrame(columns=['ID', 'selected_text'])
  for i in result:
    ID_value = i['id']
    Perdict_value = i['answer']
    temp_dict= {'ID':ID_value, 'selected_text':Perdict_value}
    temp_df = pd.DataFrame([temp_dict])
    temp=temp.append(temp_df)
  return temp


In [0]:
test_df = trans_dict_to_pd_test(predictions)

In [0]:
test_df.head()

,ID,selected_text
0,f87dea47db,last session of the day http://twitpic.com/67ezh
0,96d74cb729,exciting
0,eee518ae67,such a shame!
0,01082688c6,happy bday!
0,33987a8ee5,i like it!!


In [0]:
test_df2 = trans_dict_to_pd_test(predictions2)

In [0]:
test_df2.head()

,ID,selected_text
0,f87dea47db,last session of the day
0,96d74cb729,exciting
0,eee518ae67,such a shame!
0,01082688c6,happy bday!
0,33987a8ee5,i like it!!


In [0]:
test_df3 = trans_dict_to_pd_test(predictions3)

In [0]:
test_df3.head()

,ID,selected_text
0,f87dea47db,last session of the day
0,96d74cb729,good
0,eee518ae67,"recession hit veronique branquinho, she has to..."
0,01082688c6,happy
0,33987a8ee5,i like it!!


In [0]:
test_df.selected_text.values

array(['last session of the day http://twitpic.com/67ezh', 'exciting',
       'such a shame!', ..., 'depression...', 'i love your videos!',
       'cute'], dtype=object)

In [0]:
###adding a voting system to merge three model's result

In [0]:
my_submission = pd.DataFrame({'textID': test_data.textID, 'predicted': test_df.selected_text.values})

my_submission.head()

#you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'textID'

embedding the answer

In [0]:
def find_index(num,list1):
  index = []
  for i in range(0,len(list1)):
    if num <= list1[i]:
      index.append(i)
  return index

##### Create a voting system 
##### Vote for each characters in the sentence then select the most votes characters and frame a new perdiction

In [0]:
def votes_and_select(str1,str2,str3):
  try:
    #print(str1,str2,str3)
    s1_list = str1.split()
    s2_list = str2.split()
    s3_list = str3.split()
    length = max(len(s1_list),len(s2_list),len(s3_list))
    l = [s1_list,s2_list,s3_list]
    l2 = [str1,str2,str3]
    #print(length)
    list_of_zeros = [0]*length
    #print(list_of_zeros)
    max_list = [x for x in l if len(x) == length][0]
    other = []
    voting_list = [s1_list,s2_list,s3_list]
    voting_dict = {}
    for string in l2:
      if string in voting_dict.keys():
        voting_dict[string] = voting_dict[string]+1
      else:
        voting_dict[string] = 1
    all_values = voting_dict.values()
    max_value = max(all_values)
    #print(voting_dict)
    #print(max_value)    
    if max_value >= 2:
      final = [key for key in voting_dict if (voting_dict[key] == max_value)][0]
      if final == 'not_applicable':
        #print(final)
        for i in l2:
          if i !='not_applicable':
            final = i
        return final
      else:
        return final
    else:
      #print(max_list)
      for li in voting_list:
        #print(li)
        for s in li:
          #print(s)
          p = max_list.index(s)
          list_of_zeros[p] = list_of_zeros[p]+1
          #print(p)
          #print(list_of_zeros)
      index_list = find_index(2,list_of_zeros)
      #print(index_list)
      st = index_list[0]
      ed = index_list[-1]+1
      #print(ed)
      if st == ed:
        final_list = max_list[st]
      else:
        final_list = max_list[st:ed]
      final = ' '.join(final_list)
      return final
  except:
      return str3
    

    

In [0]:
albert_df = albert_df[['truth','Perdicted']]
roberta_df = roberta_df[['truth','Perdicted']]
distilbert_df = distilbert_df[['truth','Perdicted']]
eval_df = distilbert_df.merge(albert_df,on=['truth'],how = 'inner')
eval_df = eval_df.merge(roberta_df,on=['truth'],how = 'inner')

In [0]:
eval_df = eval_df.fillna('not_applicable')

In [0]:
eval_df['selected_text_final'] = eval_df.apply(lambda x:votes_and_select(x.Perdicted_x,x.Perdicted_y,x.Perdicted),axis=1)
eval_df

,truth,Perdicted_x,Perdicted_y,Perdicted,selected_text_final
0,resting had a whole day of walking,resting had a whole day of walking,resting had a whole day of walking,resting had a whole day of walking,resting had a whole day of walking
1,"was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ..."
2,i know! i`m so slow its horrible. don`t tell ...,horrible.,horrible.,horrible.,horrible.
3,"glad i went out, glad i didn`t leave early, an...",glad,glad,glad,glad
4,wee. done with advance audit paper,wee. done with advance audit paper,wee. done with advance audit paper,wee. done with advance audit paper,wee. done with advance audit paper
...,...,...,...,...,...
2057,sorry but there is no parking space. and i ju...,sorry,sorry,sorry,sorry
2058,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...
2059,bit excited are u bradie lol,excited,excited,excited,excited
2060,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...


In [0]:
eval_df['j_score'] = eval_df.apply(lambda x: jaccard(x.truth, x.selected_text_final), axis=1)

In [0]:
print(len(eval_df),',',eval_df.j_score.mean())

2062 , 0.7796968673640386


In [0]:
eval_df

,truth,Perdicted_x,Perdicted_y,Perdicted,selected_text_final,j_score
0,resting had a whole day of walking,resting had a whole day of walking,resting had a whole day of walking,resting had a whole day of walking,resting had a whole day of walking,1.000000
1,"was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ...","was in palawan a couple of days ago, i`ll try ...",1.000000
2,i know! i`m so slow its horrible. don`t tell ...,horrible.,horrible.,horrible.,horrible.,0.090909
3,"glad i went out, glad i didn`t leave early, an...",glad,glad,glad,glad,0.058824
4,wee. done with advance audit paper,wee. done with advance audit paper,wee. done with advance audit paper,wee. done with advance audit paper,wee. done with advance audit paper,1.000000
...,...,...,...,...,...,...
2057,sorry but there is no parking space. and i ju...,sorry,sorry,sorry,sorry,0.050000
2058,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...,nighty night twits! tty in the morning...,1.000000
2059,bit excited are u bradie lol,excited,excited,excited,excited,0.166667
2060,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...,is pretty hammered drunk! props to me! two nig...,1.000000


In [0]:
eval_df.to_csv('123.csv')

In [0]:
test_df = test_df.merge(test_df2,on=['ID'],how = 'inner')
test_df = test_df.merge(test_df3,on=['ID'],how = 'inner')

In [0]:
test_df['selected_text_final'] = test_df.apply(lambda x:votes_and_select(x.selected_text_x,x.selected_text_y,x.selected_text),axis=1)

In [0]:
my_submission = pd.DataFrame({'textID': test_data.textID, 'predicted': test_df.selected_text_final.values})

my_submission.head()

#you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

,textID,predicted
0,f87dea47db,last session of the day
1,96d74cb729,exciting
2,eee518ae67,such a shame!
3,01082688c6,happy bday!
4,33987a8ee5,i like it!!
